In [1]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from collections import Counter

# Load the Data

In [3]:
d = pd.read_json('tmp/raw_data.json')
d.index.name = 'id'
d = d[['features']]
print len(d)

124011


In [4]:
d.dtypes

features    object
dtype: object

In [5]:
d.features.head()

id
0       [Elevator, Laundry in Building, Laundry in Uni...
1                   [Pre-War, Dogs Allowed, Cats Allowed]
10                                                     []
100                           [Doorman, Elevator, No Fee]
1000    [Roof Deck, Balcony, Elevator, Laundry in Buil...
Name: features, dtype: object

# Clean Features

In [6]:
d.features = d.features.apply(lambda fl: [f.lower().replace(' ', '_') for f in fl])

# Number of Features

In [7]:
d['num_features'] = d.features.apply(lambda f: len(f))

# Most Frequent Features

In [8]:
flat_features = [f for apt in d.features.tolist() for f in apt]
flat_feature_counts = Counter(flat_features)
flat_feature_counts.most_common(20)

[(u'elevator', 65833),
 (u'cats_allowed', 59194),
 (u'hardwood_floors', 59155),
 (u'dogs_allowed', 55207),
 (u'doorman', 52505),
 (u'dishwasher', 52035),
 (u'laundry_in_building', 47483),
 (u'no_fee', 45450),
 (u'fitness_center', 33420),
 (u'laundry_in_unit', 23752),
 (u'pre-war', 23111),
 (u'roof_deck', 16466),
 (u'outdoor_space', 13414),
 (u'dining_room', 12847),
 (u'high_speed_internet', 10622),
 (u'balcony', 7793),
 (u'swimming_pool', 7153),
 (u'new_construction', 6457),
 (u'terrace', 5707),
 (u'exclusive', 5470)]

In [9]:
top_10_features = set([fc[0] for fc in flat_feature_counts.most_common(10)])
top_30_features = set([fc[0] for fc in flat_feature_counts.most_common(30)])

In [10]:
d['top_10_feature_coverage'] = d.features.apply(
    lambda fs: len(top_10_features.intersection(set(fs)))/10.0)
d['top_30_feature_coverage'] = d.features.apply(
    lambda fs: len(top_30_features.intersection(set(fs)))/30.0)
d['top_10_feature_ratio'] = d.features.apply(
    lambda fs: len(top_10_features.intersection(set(fs)))/(1.0 * len(fs)) if len(fs) > 0 else 0)
d['top_30_feature_ratio'] = d.features.apply(
    lambda fs: len(top_30_features.intersection(set(fs)))/(1.0 * len(fs)) if len(fs) > 0 else 0)

In [12]:
d.head(10)

,num_features,top_10_feature_coverage,top_30_feature_coverage,top_10_feature_ratio,top_30_feature_ratio
id,,,,,
0,6,0.5,0.200000,0.833333,1.000000
1,3,0.2,0.100000,0.666667,1.000000
10,0,0.0,0.000000,0.000000,0.000000
100,3,0.3,0.100000,1.000000,1.000000
1000,10,0.5,0.333333,0.500000,1.000000
10000,5,0.5,0.166667,1.000000,1.000000
100000,14,0.9,0.433333,0.642857,0.928571
100001,11,0.9,0.366667,0.818182,1.000000
100002,2,0.1,0.066667,0.500000,1.000000


# Save the Data

In [13]:
d.to_csv('tmp/features_features.csv')